In [ ]:
!git clone https://github.com/tdw75/research-seminar-point-cloud-data-analysis.git

In [ ]:
import torch

!pip uninstall torch-scatter torch-sparse torch-geometric torch-cluster  --y
!pip install torch-scatter -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-cluster -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install git+https://github.com/pyg-team/pytorch_geometric.git

In [ ]:
import sys
sys.path.insert(0, '/content/research-seminar-point-cloud-data-analysis')

In [ ]:
!mkdir data_files
!mkdir data_files/saved_weights

## Import packages

In [12]:
from datetime import datetime

import torch
import torch_geometric.transforms as T
from torch_geometric import seed_everything

from src.data.dataset import init_train_data_loader, init_test_data_loaders
from src.modelling.pointnet_plusplus import PointNetPlusPlus
from src.modelling.training import train_model

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# configuration
num_points = 2048
num_classes = 10
num_epochs = 200
num_workers = 2

## Load data

In [7]:
root = "../../data_files"
seed_everything(0)
pre_transform, transform = T.NormalizeScale(), T.SamplePoints(64)
train_loader = init_train_data_loader(
    f"{root}/modelnet", num_points, num_classes, batch_size=32, shuffle=True, num_workers=num_workers
)
seed_everything(0)
test_loaders = init_test_data_loaders(
    f"{root}/modelnet", train_loader, num_classes, is_affine_transformations=True
)

In [11]:
saved_weights = None
net = PointNetPlusPlus()

if saved_weights:
    path = f"{root}/saved_weights/weights_{saved_weights}.pth"
    net.load_state_dict(torch.load(path))
else:
    dt = datetime.now().replace(microsecond=0)
    path = f"{root}/saved_weights/weights_{dt}_{num_classes}_{num_points}_{num_epochs}.pth"
    train_model(net, train_loader, test_loaders["original"], n_epochs=num_epochs)
    torch.save(net.state_dict(), path)

## Test

In [13]:
from src.modelling.training import val_step

for key in test_loaders:
    print(key, val_step(net, test_loaders[key], device))

original 0.8755506607929515
flipped 0.6189427312775331
rotated 0.44383259911894274
scaled 0.236784140969163
